In [85]:
%run ./seg_fraud.ipynb

DONE, SET LENGTH: 3234


In [86]:
%run ./seg_normal.ipynb

DONE, SET LENGTH: 13265


In [87]:
f1 = open('fraud.txt', 'r')
f2 = open('enron.txt', 'r')

corpus_one = [item for item in f1]
corpus_two = [item for item in f2]

fraud_set = split_set(corpus_one)
norm_set = split_enron_set(corpus_two)

In [88]:
fraud_set = fraud_set[:6]
norm_set = norm_set[:6]

In [89]:
def label(fraud, norm):
    new_set = []
    for item in fraud:
        t = (item, 'fraud')
        new_set.append(t)
    for item in norm:
        t = (item, 'norm')
        new_set.append(t)
    return new_set    

In [90]:
x = label(fraud_set,norm_set)

#NECESSARY IMPORTS:
#torch
#pytorch-pretrained-bert
#torchnlp

In [91]:
import pandas as pd 
import numpy as np 
import torch.nn as nn
from pytorch_pretrained_bert import BertTokenizer, BertModel
import torch
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report
import torch.utils.data

In [92]:
g = pd.DataFrame(x)

In [93]:
pd.set_option('display.max_columns', None)
df = g
print(df.head())


                                                   0      1
0  Return-Path: <james_ngola2002@maktoob.com>\nX-...  fraud
1  Return-Path: <bensul2004nng@spinfinder.com>\nX...  fraud
2  \nFROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF ...  fraud
3  \nFROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF ...  fraud
4  \nDear sir, \n \nIt is with a heart full of ho...  fraud


In [94]:
from collections import Counter
print(Counter(df[1].values))

Counter({'fraud': 6, 'norm': 6})


In [95]:
df.dropna(inplace = True)


In [96]:
df = df.rename(columns={0: "text", 1: "type"})

In [97]:
df = df.replace('pos', 'fraud')
df = df.replace('neg', 'norm')


In [98]:
df.head()

,text,type
0,Return-Path: <james_ngola2002@maktoob.com>\nX-...,fraud
1,Return-Path: <bensul2004nng@spinfinder.com>\nX...,fraud
2,\nFROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF ...,fraud
3,\nFROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF ...,fraud
4,"\nDear sir, \n \nIt is with a heart full of ho...",fraud


In [99]:
df_fake = df[df['type'] == 'fraud'] 
df_statire = df[df['type'] == 'norm'] 
df_statire = df_statire.sample(n=len(df_fake))
df = df_statire.append(df_fake)
df = df.sample(frac=1, random_state = 24).reset_index(drop=True)

In [100]:
train_data = df.head(6)
test_data = df.tail(6)

In [101]:
train_data.head()

,text,type
0,\nATTENTION: ...,fraud
1,"\nRandy,\n\n Can you send me a schedule of the...",norm
2,\nFROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF ...,fraud
3,"\nDear sir, \n \nIt is with a heart full of ho...",fraud
4,"""file"",""message""\n""allen-p/_sent_mail/1."",""Mes...",norm


In [102]:
test_data.head()

,text,type
6,\nFROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF ...,fraud
7,"\nHere is our forecast\n\n ""\n""allen-p/_sent_m...",norm
8,Return-Path: <bensul2004nng@spinfinder.com>\nX...,fraud
9,"\ntest successful. way to go!!!""\n""allen-p/_s...",norm
10,"\nLet's shoot for Tuesday at 11:45. ""\n""allen...",norm


In [103]:
train_data = [{'text': text, 'type': type_data } for text in list(train_data['text']) for type_data in list(train_data['type'])]
test_data = [{'text': text, 'type': type_data } for text in list(test_data['text']) for type_data in list(test_data['type'])]

In [104]:
train_texts, train_labels = list(zip(*map(lambda d: (d['text'], d['type']), train_data)))
test_texts, test_labels = list(zip(*map(lambda d: (d['text'], d['type']), test_data)))

In [105]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
train_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:511], train_texts))
test_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:511], test_texts))
train_tokens_ids = list(map(tokenizer.convert_tokens_to_ids, train_tokens))
test_tokens_ids = list(map(tokenizer.convert_tokens_to_ids, test_tokens))
train_tokens_ids = pad_sequences(train_tokens_ids, maxlen=512, truncating="post", padding="post", dtype="int")
test_tokens_ids = pad_sequences(test_tokens_ids, maxlen=512, truncating="post", padding="post", dtype="int")

In [106]:
print(len(train_labels))
print(type(train_labels))
for item in train_labels:(print(item))

36
<class 'tuple'>
fraud
norm
fraud
fraud
norm
fraud
fraud
norm
fraud
fraud
norm
fraud
fraud
norm
fraud
fraud
norm
fraud
fraud
norm
fraud
fraud
norm
fraud
fraud
norm
fraud
fraud
norm
fraud
fraud
norm
fraud
fraud
norm
fraud


In [107]:
print(len(test_labels))
print(type(test_labels))
for item in test_labels: print(item)

36
<class 'tuple'>
fraud
norm
fraud
norm
norm
norm
fraud
norm
fraud
norm
norm
norm
fraud
norm
fraud
norm
norm
norm
fraud
norm
fraud
norm
norm
norm
fraud
norm
fraud
norm
norm
norm
fraud
norm
fraud
norm
norm
norm


In [108]:
train_y = []
test_y = []

t1 = []
t2 = []

for item in train_labels:
    if str(item) == 'fraud': t1.append(1)
    else: t1.append(0)    

        

for item in test_labels:
    if str(item) == 'fraud': t2.append(1)
    else: t2.append(0)        

#train_y_step = np.array(train_labels) == 'fake'
#test_y_step = np.array(test_labels) == 'fake'



#for i in range(len(train_y_step)):
    #train_y.append(int(train_y_step[i]))
#for i in range(len(test_y_step)):
    #test_y.append(int(test_y_step[i]))
    
train_y = np.array(t1)
test_y = np.array(t2)

In [109]:
for item in train_y: print(item)

1
0
1
1
0
1
1
0
1
1
0
1
1
0
1
1
0
1
1
0
1
1
0
1
1
0
1
1
0
1
1
0
1
1
0
1


In [110]:
for item in test_y: print(item)

1
0
1
0
0
0
1
0
1
0
0
0
1
0
1
0
0
0
1
0
1
0
0
0
1
0
1
0
0
0
1
0
1
0
0
0


In [111]:
class BertBinaryClassifier(nn.Module):
    def __init__(self, dropout=0.1):
        super(BertBinaryClassifier, self).__init__()
    
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, tokens, masks=None):
        _, pooled_output = self.bert(tokens, attention_mask=masks, output_all_encoded_layers=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        proba = self.sigmoid(linear_output)
        return proba

In [112]:
train_masks = [[float(i > 0) for i in ii] for ii in train_tokens_ids]
test_masks = [[float(i > 0) for i in ii] for ii in test_tokens_ids]
train_masks_tensor = torch.tensor(train_masks)
test_masks_tensor = torch.tensor(test_masks)

In [113]:
train_tokens_tensor = torch.tensor(train_tokens_ids)
train_y_tensor = torch.tensor(train_y.reshape(-1, 1)).float()
test_tokens_tensor = torch.tensor(test_tokens_ids)
test_y_tensor = torch.tensor(test_y.reshape(-1, 1)).float()

In [114]:
train_dataset =  torch.utils.data.TensorDataset(train_tokens_tensor, train_masks_tensor, train_y_tensor)
train_sampler =  torch.utils.data.RandomSampler(train_dataset)
train_dataloader =  torch.utils.data.DataLoader(train_dataset, sampler=train_sampler, batch_size=1)
test_dataset =  torch.utils.data.TensorDataset(test_tokens_tensor, test_masks_tensor, test_y_tensor)
test_sampler =  torch.utils.data.SequentialSampler(test_dataset)
test_dataloader =  torch.utils.data.DataLoader(test_dataset, sampler=test_sampler, batch_size=1)

STATEMENT BELOW ITERATES THROUGH EACH ITEM IN THE TRAINING DATASET AT ONE EPOCH

In [115]:
BATCH_SIZE = 1
EPOCHS = 1
bert_clf = BertBinaryClassifier()
optimizer = torch.optim.Adam(bert_clf.parameters(), lr=3e-6)
for epoch_num in range(EPOCHS):
    bert_clf.train()
    train_loss = 0
    for step_num, batch_data in enumerate(train_dataloader):
        token_ids, masks, labels = tuple(t for t in batch_data)
        probas = bert_clf(token_ids, masks)
        loss_func = nn.BCELoss()
        batch_loss = loss_func(probas, labels)
        train_loss += batch_loss.item()
        bert_clf.zero_grad()
        batch_loss.backward()
        optimizer.step()
        print('Epoch: ', epoch_num + 1)
        print("\r" + "{0}/{1} loss: {2} ".format(step_num, len(train_data) / BATCH_SIZE, train_loss / (step_num + 1)))

Epoch:  1
0/36.0 loss: 0.7049091458320618 
Epoch:  1
1/36.0 loss: 0.7767441272735596 
Epoch:  1
2/36.0 loss: 0.7959642012914022 
Epoch:  1
3/36.0 loss: 0.7931198626756668 
Epoch:  1
4/36.0 loss: 0.7663469314575195 
Epoch:  1
5/36.0 loss: 0.7504710058371226 
Epoch:  1
6/36.0 loss: 0.7454165816307068 
Epoch:  1
7/36.0 loss: 0.7371055707335472 
Epoch:  1
8/36.0 loss: 0.7291107641326057 
Epoch:  1
9/36.0 loss: 0.7330771625041962 
Epoch:  1
10/36.0 loss: 0.7316584261980924 
Epoch:  1
11/36.0 loss: 0.7363427529732386 
Epoch:  1
12/36.0 loss: 0.7330259405649625 
Epoch:  1
13/36.0 loss: 0.7447047914777484 
Epoch:  1
14/36.0 loss: 0.7438952883084615 
Epoch:  1
15/36.0 loss: 0.7427257150411606 
Epoch:  1
16/36.0 loss: 0.7464038379052106 
Epoch:  1
17/36.0 loss: 0.7456305060121748 
Epoch:  1
18/36.0 loss: 0.7492861214436983 
Epoch:  1
19/36.0 loss: 0.7436093747615814 
Epoch:  1
20/36.0 loss: 0.7414100964864095 
Epoch:  1
21/36.0 loss: 0.7394105819138613 
Epoch:  1
22/36.0 loss: 0.7381922654483629

In [116]:
bert_clf.eval()
bert_predicted = []
all_logits = []
with torch.no_grad():
    for step_num, batch_data in enumerate(test_dataloader):
        token_ids, masks, labels = tuple(t for t in batch_data)
        logits = bert_clf(token_ids, masks)
        loss_func = nn.BCELoss()
        loss = loss_func(logits, labels)
        numpy_logits = logits.cpu().detach().numpy()
        
        bert_predicted += list(numpy_logits[:, 0] > 0.5)
        all_logits += list(numpy_logits[:, 0])
        
print(classification_report(test_y, bert_predicted))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00        24
           1       0.33      1.00      0.50        12

   micro avg       0.33      0.33      0.33        36
   macro avg       0.17      0.50      0.25        36
weighted avg       0.11      0.33      0.17        36



/home/gcurtiss/venv/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
